# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [19]:
#Your code here
import pandas as pd

df = pd.read_csv("homepage_actions.csv")

print(df.head())
print(df.info())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


In [20]:
# Count of clicks
clicks = df[df["action"] == "click"].id.nunique()
print(f"Viewers who clicked: {clicks}")

Viewers who clicked: 1860


In [21]:
# Viewers and clickers
viewers = df[df["action"] == "view"].id.unique()
clickers = df[df["action"] == "click"].id.unique()

# Clicks without views
clicks_without_views = set(clickers) - set(viewers)
print(f"Clicks without views: {len(clicks_without_views)}")

Clicks without views: 0


In [22]:
# Find users in both control and experiment
control_ids = df[df["group"] == "control"].id
experiment_ids = df[df["group"] == "experiment"].id

overlap = set(control_ids) & set(experiment_ids)
print(f"Overlapping users: {len(overlap)}")

Overlapping users: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [23]:
#Your code here
# Pivot table of click rate per group
click_rates = df.pivot_table(index='group', columns='action', aggfunc='count', values='id')
click_rates['click_rate'] = click_rates['click'] / (click_rates['click'] + click_rates['view'])
print(click_rates)

action      click  view  click_rate
group                              
control       932  3332    0.218574
experiment    928  2996    0.236493


In [24]:
from statsmodels.stats.proportion import proportions_ztest

# Get counts
control = df[df['group'] == 'control']
experiment = df[df['group'] == 'experiment']

clicks_control = control[control['action'] == 'click'].id.nunique()
views_control = control.id.nunique()

clicks_experiment = experiment[experiment['action'] == 'click'].id.nunique()
views_experiment = experiment.id.nunique()

count = [clicks_control, clicks_experiment]
nobs = [views_control, views_experiment]

zstat, pval = proportions_ztest(count, nobs)
print(f"Z-stat: {zstat}, P-value: {pval}")

Z-stat: -2.618563885349469, P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [25]:
#Your code here
ctr_control = clicks_control / views_control
expected_clicks_experiment = ctr_control * views_experiment
print(f"Expected Clicks (Exp Group): {expected_clicks_experiment}")

Expected Clicks (Exp Group): 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [26]:
#Your code here
actual_clicks_exp = clicks_experiment
std_dev = (views_experiment * ctr_control * (1 - ctr_control)) ** 0.5

z = (actual_clicks_exp - expected_clicks_experiment) / std_dev
print(f"Z-score: {z}")

Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [27]:
#Your code here
from scipy.stats import norm

p_manual = 1 - norm.cdf(z)
print(f"P-value (manual): {p_manual}")

P-value (manual): 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The experimental homepage has a statistically significant effect on clicks.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.